# 02. Azure ML Pipeline Creation - AutoML for Time-Series Forecasting
This notebook demonstrates creation of an Azure ML pipeline designed to load data from an AML-linked Datastore, split into train/forecasting datasets, submit an AutoML job, and then register the model into the workspace and save a forward-looking forecast. <i>Run this notebook after running </i>`01_Setup_AML_Env.ipynb`.

### Import required packages

In [ ]:
from azureml.core import Workspace, Experiment, Datastore, Environment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute, DataFactoryCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.pipeline.core import Pipeline, PipelineParameter, PipelineData
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import PipelineParameter, PipelineData
from azureml.data.output_dataset_config import OutputTabularDatasetConfig, OutputDatasetConfig, OutputFileDatasetConfig
from azureml.data.datapath import DataPath
from azureml.data.data_reference import DataReference
from azureml.data.sql_data_reference import SqlDataReference
from azureml.pipeline.steps import DataTransferStep
import logging

### Connect to AML workspace and get reference to training compute

In [ ]:
ws = Workspace.from_config()
cluster_name = 'cpucluster'
compute_target = ComputeTarget(workspace=ws, name=cluster_name)

### Create Run Configuration
The `RunConfiguration` defines the environment used across all python steps. You can optionally add additional conda or pip packages to be added to your environment. [More details here](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.conda_dependencies.condadependencies?view=azure-ml-py).

Here, we are using an existing conda environment yaml file (`./automl_env.yml`) to create our environment and also register it to the AML workspace so that it can be used for future forecasting operations.

In [ ]:
run_config = RunConfiguration()
run_config.docker.use_docker = True
run_config.environment = Environment.from_conda_specification(file_path='automl_env.yml', name='AutoMLEnv')
run_config.environment.register(ws)

### Get reference to default datastore

In [ ]:
default_ds = ws.get_default_datastore()

### Define Input and Output Datasets
Retrieve references to two registered datasets to be consumed as inputs by the pipeline, and configure an `OutputFileDatasetConfig` to point to a location in blob storage where the pipeline output will be written. 

In [ ]:
from azureml.data import DataType
column_dictionary = {
    'Store':DataType.to_string(),
    'Brand':DataType.to_string(),
    'Quantity':DataType.to_long(),
    'Advert':DataType.to_string(),
    'Price':DataType.to_float(),
    'Revenue':DataType.to_float(),
}

training_data = OutputFileDatasetConfig(name='Training_Data', destination=(default_ds, 'training_data/{run-id}')).read_delimited_files(set_column_types=column_dictionary)
forecasting_data = OutputFileDatasetConfig(name='Forecasting_Data', destination=(default_ds, 'forecasting_data/{run-id}')).read_delimited_files(set_column_types=column_dictionary)

### Define Pipeline Parameters
`PipelineParameter` objects serve as variable inputs to an Azure ML pipeline and can be specified at runtime. Below we define the following parameters for our Azure ML Pipeline:

| Parameter Name | Parameter Description |
|----------------|-----------------------|
| `datastore_relative_path` | Relative path on the Azure ML-linked datastore where source time-series data can be retrieved. |
| `raw_dataset_name` | Dataset name to be used when registering raw data into the AML workspace. |
| `train_dataset_name` | Dataset name to be used when registering training data into the AML workspace. |
| `forecast_dataset_name` | Dataset name to be used when registering forecasting data (data that has not yet been submitted to the model for predictions) into the AML workspace. |
| `result_dataset_name` | Dataset name to be used when registering forecast results data into the AML workspace. |
| `timestamp_column` | Column which contains timestamp values in your source dataset. |
| `cutoff_date` | Date to be used to splitting data into train/forecast subsets. |
| `model_name` | Name to be assigned to trained model prior to registering in the datastore. |

[PipelineParameter](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-core/azureml.pipeline.core.graph.pipelineparameter?view=azure-ml-py)

In [ ]:
datastore_relative_path = PipelineParameter(name='datastore_relative_path', default_value='sample_data')
raw_dataset_name = PipelineParameter(name='raw_dataset_name', default_value='Raw_Sample_Dataset')
train_dataset_name = PipelineParameter(name='train_dataset_name', default_value='Train_Sample_Dataset')
forecast_dataset_name = PipelineParameter(name='forecast_dataset_name', default_value='Forecast_Sample_Dataset')
result_dataset_name = PipelineParameter(name='result_dataset_name', default_value='Result_Sample_Dataset')
timestamp_column = PipelineParameter(name='timestamp_column', default_value='WeekStarting')
cutoff_date = PipelineParameter(name='cutoff_date', default_value='1992-05-28')
model_name = PipelineParameter(name='model_name', default_value='Sample_Forecasting_Model')

### Define Pipeline Steps
The pipeline below consists of three distinct steps to prepare data, train models, and generate a forecast/register the model to the workspace. First, we call `organize_data.py` and retrieve data from the registered datastore, split into training and forecasting subsets based on the specified `cutoff_date`, save each time-series to a file and register as a new File Dataset. 

From here we configure an AutoML job forecasting job which will train and return the best performing model for your particular time-series.

Following training, we generate a forecast across the dates included in the `forecast_dataset` using the best-performing model.

Finally, we aggregate all of the forecasted results across time-series into a single dataset (`result_dataset`) and register that in the AML datastore.

In [ ]:
#Create PythonScriptStep to gather data from remote source and register as AML dataset
organize_data_step = PythonScriptStep(
    name='Organize Time-Series Data',
    script_name="get_data.py", 
    arguments=["--raw_dataset_name", raw_dataset_name, 
               "--train_dataset_name", train_dataset_name, 
               "--forecast_dataset_name", forecast_dataset_name, 
               "--datastore_relative_path", datastore_relative_path, 
              '--timestamp_column', timestamp_column,
              '--cutoff_date', cutoff_date,
              '--training_data', training_data,
              '--forecasting_data', forecasting_data],
    outputs=[training_data, forecasting_data],
    compute_target=compute_target, 
    source_directory='./pipeline_step_scripts',
    allow_reuse=False,
    runconfig=run_config
)

from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep
automl_config = AutoMLConfig(
    task= 'forecasting',
    primary_metric= 'normalized_root_mean_squared_error',
    iteration_timeout_minutes= 60,
    iterations = 30,
    experiment_timeout_hours= 3,
    label_column_name= 'Quantity',
    n_cross_validations= 3,
    debug_log='automl_sales_debug.txt',
    time_column_name= 'WeekStarting',
    max_horizon = 20,
    compute_target = compute_target,
    training_data=training_data)

train_model_step = AutoMLStep(name='Train Forecasting Model (AutoML)',
    automl_config=automl_config,
    passthru_automl_config=False,
    enable_default_model_output=False,
    enable_default_metrics_output=False,
    allow_reuse=False)

# #Evaluate and register
register_step = PythonScriptStep(
    name = 'Register Model and Generate Forecast',
    script_name='register.py',
    inputs=[forecasting_data.as_input(name='Forecasting_Data')],
    arguments=['--model_name', model_name, '--target_column', 'Quantity', '--result_dataset_name', result_dataset_name],
    compute_target=compute_target,
    source_directory='./pipeline_step_scripts',
    allow_reuse=False,
    runconfig=run_config
)
register_step.run_after(train_model_step)

### Create Pipeline
Pipelines are reusable in AML workflows that can be triggered in multiple ways (manual, programmatic, scheduled, etc.) Create an Azure ML Pipeline by specifying the pipeline steps to be executed.

[What are Machine Learning Pipelines?](https://docs.microsoft.com/en-us/azure/machine-learning/concept-ml-pipelines)

In [ ]:
pipeline = Pipeline(workspace=ws, steps=[organize_data_step, train_model_step, register_step])

### Create Published PipelineEndpoint
`PipelineEndpoints` can be used to create a versions of published pipelines while maintaining a consistent endpoint. These endpoint URLs can be triggered remotely by submitting an authenticated request and updates to the underlying pipeline are tracked in the AML workspace.

[PipelineEndpoint](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-core/azureml.pipeline.core.pipeline_endpoint.pipelineendpoint?view=azure-ml-py)

In [ ]:
from azureml.pipeline.core import PipelineEndpoint

def published_pipeline_to_pipeline_endpoint(
    workspace,
    published_pipeline,
    pipeline_endpoint_name,
    pipeline_endpoint_description="AML Pipeline for training forecasting models using AutoML.",
):
    try:
        pipeline_endpoint = PipelineEndpoint.get(
            workspace=workspace, name=pipeline_endpoint_name
        )
        print("using existing PipelineEndpoint...")
        pipeline_endpoint.add_default(published_pipeline)
    except Exception as ex:
        print(ex)
        # create PipelineEndpoint if it doesn't exist
        print("PipelineEndpoint does not exist, creating one for you...")
        pipeline_endpoint = PipelineEndpoint.publish(
            workspace=workspace,
            name=pipeline_endpoint_name,
            pipeline=published_pipeline,
            description=pipeline_endpoint_description
        )


pipeline_endpoint_name = 'Time-Series Forecast Model Training'
pipeline_endpoint_description = 'AML Pipeline for training forecasting models using AutoML'

published_pipeline = pipeline.publish(name=pipeline_endpoint_name,
                                     description=pipeline_endpoint_description,
                                     continue_on_step_failure=False)

published_pipeline_to_pipeline_endpoint(
    workspace=ws,
    published_pipeline=published_pipeline,
    pipeline_endpoint_name=pipeline_endpoint_name,
    pipeline_endpoint_description=pipeline_endpoint_description
)

### Optional: Trigger a Pipeline execution from the notebook
You can create an Experiment (logical collection for runs) and submit a pipeline run directly from this notebook by running the commands below.

In [ ]:
experiment = Experiment(ws, 'sample-automl-forecasting-run')
run = experiment.submit(pipeline)
run.wait_for_completion(show_output=True)